PROCESS DOCUMENT

In [ ]:
import os
import json
from IPython.display import Markdown
from phi.agent import Agent
from phi.model.ollama import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from utils.document_processor import DocumentProcessor  

# Inisialisasi path dan model
DATA_PATH = "./data"
INDEX_PATH = "faiss_index"
CHUNKED_DATA_PATH = "./chunked_data"  
METADATA_PATH = "./metadata"  
OLLAMA_MODEL = "llama3.2"

# Pastikan folder tersedia
os.makedirs(CHUNKED_DATA_PATH, exist_ok=True)
os.makedirs(METADATA_PATH, exist_ok=True)

# Inisialisasi model Ollama
llm = Ollama(id=OLLAMA_MODEL)

# Inisialisasi embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Inisialisasi document processor
docs = DocumentProcessor()

# Fungsi untuk chunking teks
def chunk_text(text: str):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200, length_function=len, separators=["\n\n", "\n", " "]
    )
    return text_splitter.split_text(text)

# Inisialisasi Agent dengan model Ollama
agent = Agent(model=llm, show_tool_calls=True, markdown=True)

# Langkah 1: Proses dokumen dengan agentic chunking
extracted_docs = []
for filename in os.listdir(DATA_PATH):
    valid_extensions = ('.pdf', '.docx', '.txt')
    if not filename.lower().endswith(valid_extensions):
        continue

    filepath = os.path.join(DATA_PATH, filename)

    try:
        # Gunakan DocumentProcessor untuk membaca file
        with open(filepath, "rb") as f:
            document = f.read()
            result = docs.process_document(document, filename)

        if result is None:
            print(f"Gagal memproses {filename}, melewati file ini.")
            continue

        plain_text = result[3]  # Pastikan indeks benar

        print(f"[INFO] {filename} - Panjang teks sebelum pemrosesan: {len(plain_text)}")

        # **Pecah dokumen menjadi bagian lebih kecil sebelum dikirim ke Agent**
        text_chunks = chunk_text(plain_text)
        structured_chunks = []

        for i, chunk in enumerate(text_chunks):
            print(f"[INFO] Mengirim chunk {i+1} ke agent (panjang: {len(chunk)})")
            response = agent.run(
                f"Organize the following text into well-structured segments:\n{chunk}",
                max_tokens=4000
            )

            # Ambil teks dari response
            if isinstance(response, str):
                response_text = response
            elif isinstance(response, dict):
                response_text = response.get("text", "")
            else:
                response_text = response.content if hasattr(response, "content") else str(response)

            response_text = response_text.strip()
            structured_chunks.append(response_text)

        # **Gabungkan kembali hasil agent**
        final_text = "\n\n".join(structured_chunks)

        print(f"[INFO] {filename} - Panjang teks setelah agent: {len(final_text)}")

        # Lakukan chunking ulang setelah agent
        final_chunks = chunk_text(final_text)

        # Simpan hasil chunking dalam list
        chunk_data = [{"chunk_id": i+1, "text": chunk} for i, chunk in enumerate(final_chunks)]
        metadata = {"filename": filename, "total_chunks": len(final_chunks)}

        extracted_docs.extend([Document(page_content=chunk["text"], metadata={"chunk_id": chunk["chunk_id"], **metadata}) for chunk in chunk_data])

        # Simpan hasil chunking dalam file txt
        chunked_filepath = os.path.join(CHUNKED_DATA_PATH, f"chunked_{filename}.txt")
        with open(chunked_filepath, "w", encoding="utf-8") as chunked_file:
            for chunk in chunk_data:
                chunked_file.write(f"Chunk {chunk['chunk_id']}:\n")
                chunked_file.write(f"{chunk['text']}\n")
                chunked_file.write("\n---\n\n")  

        # Simpan metadata dalam file JSON
        metadata_filepath = os.path.join(METADATA_PATH, f"metadata_{filename}.json")
        with open(metadata_filepath, "w", encoding="utf-8") as metadata_file:
            json.dump(metadata, metadata_file, indent=4)

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Simpan ke FAISS secara lokal
vector_store = FAISS.from_documents(extracted_docs, embedding_model)
vector_store.save_local(INDEX_PATH)

print(f"Total chunks generated for {filename}: {len(final_chunks)}")
print("Proses chunking selesai. Hasilnya disimpan dalam folder 'chunked_data' dan metadata di 'metadata'.")


[INFO] Dokumen (1).pdf - Panjang teks sebelum pemrosesan: 3986
[INFO] Mengirim chunk 1 ke agent (panjang: 1000)
[INFO] Mengirim chunk 2 ke agent (panjang: 999)
[INFO] Mengirim chunk 3 ke agent (panjang: 998)
[INFO] Mengirim chunk 4 ke agent (panjang: 945)
[INFO] Mengirim chunk 5 ke agent (panjang: 631)
[INFO] Dokumen (1).pdf - Panjang teks setelah agent: 5410
Total chunks generated for Dokumen (1).pdf: 7
Proses chunking selesai. Hasilnya disimpan dalam folder 'chunked_data' dan metadata di 'metadata'.


In [62]:
# Query dengan RAG
query = "Apa isi dokumen tentang topik X?"
query_embedding = embedding_model.embed_query(query)
retrieved_docs = vector_store.similarity_search_by_vector(query_embedding, k=3)

# Gabungkan dokumen hasil pencarian untuk input ke model
retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

TEST OLLAMA MODEL

In [63]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [64]:
OLLAMA_MODEL = "llama3.2"
COLLECTION_NAME = "ollama_vectore_test"

In [65]:
template = """
  Anda adalah asisten yang membantu dalam meringkas teks.  
  Hanya sertakan informasi yang ada dalam dokumen.  
  Jangan tambahkan opini atau analisis Anda sendiri.  

  Dokumen:  
  "{document}"  
  Ringkasan:  
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model=OLLAMA_MODEL)

chain = prompt | model

response = chain.invoke({"document": plain_text})
Markdown(response)


Berikut ringkasan dokumen:

PT Pertamina (Persero) adalah perusahaan energi nasional yang didirikan pada tahun 1950-an sebagai PT Eksploitasi Tambang Minyak Sumatera Utara. Pada tahun 1961, nama perusahaan diubah menjadi Perusahaan Negara (PN) dengan nama PN Pertambangan Minyak Nasional (Permina). Pada tahun 1971, PN Permina bergabung dengan PN Pertamin untuk membentuk PN Pertambangan Minyak dan Gas Bumi Negara (Pertamina). Pada tahun 2003, nama perusahaan diubah menjadi PT Pertamina (Persero).

Pada tahun 2007, PERTAMINA mengubah visi perusahaan menjadi "Menjadi Perusahaan Minyak Nasional Kelas Dunia". Pada tahun 2011, visi tersebut disempurnakan menjadi "Menjadi Perusahaan Energi Nasional Kelas Dunia".

Pada tahun 2017, PERTAMINA menuntaskan akuisisi 72,65% saham perusahaan migas Prancis Maurel et Prom (M&P). Pada tahun 2018, PT Pertamina Gas (Pertagas) bergabung dengan PT Perusahaan Gas Negara (PGN), sehingga PERTAMINA memantapkan posisinya sebagai garda terdepan yang menjaga ketahanan energi nasional.

Pada tahun 2020 dan 2021, PERTAMINA melaksanakan restrukturisasi pembentukan subholding baru, termasuk Upstream Subholding, Gas Subholding, Refinery and Petrochemical Subholding, Power & NRE Subholding, Commercial and Trading Subholding, dan Integrated Marine Logistics Subholding.

TEST OPENAI MODEL

In [ ]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-35-turbo"
response = openai.ChatCompletion.create(
  engine=model,
  messages=[
    {
      "role": "system", 
      "content": """
        You are a helpful assistant for text summarization.
        Only include information that is part of the document. 
        Do not include your own opinion or analysis.
      """
    },
    {
      "role": "user",
      "content": plain_text
    }
  ],
)

Markdown(response.choices[0].message.content)